# Additive Secret Sharing

Author: 
- Carlos Salgado - [email](mailto:csalgado@uwo.ca) - [linkedin](https://www.linkedin.com/in/eng-socd/) - [github](https://github.com/socd06)

## Additive Secret Sharing
Additive Secret Sharing is a mechanism to share data among parties and to perform computation on it. 

![Secret Sharing](img/secret-sharing.jpg)

## Sharing
A secret `s` is uniformly split into `n` shares, one per shareholder (also known as worker, node, user or party) using some randomness `r`, also known as some **very high random prime** number `Q`. 

$ F_s (s, r, n) = ( s_1, s_2, ..., s_n ) $

## Reconstruction
`s` can be reconstructed (decrypted) by adding up **all the shares** and taking the [*modulo*](https://en.wikipedia.org/wiki/Modulo_operation) of the random prime number `Q`, used to encrypt the shares originally. 

$ s = ( \: \sum \limits _{i=1} ^n s_i \: ) \; mod \; Q $

## 32-bit Integer Secrets
A secret is the data or message that a party wants to secure. In additive secret sharing, secrets (and therefore, shares) must be members of a fixed [finite field](https://en.wikipedia.org/wiki/Finite_field). Particularly, the literature mentions shares should be members of the $ {\mathbb{Z}_{2^{32}}} $ [ring](https://en.wikipedia.org/wiki/Ring_(mathematics)), which is the [ring of integers](https://en.wikipedia.org/wiki/Ring_of_integers) that fit within [32-bits](https://en.wikipedia.org/wiki/32-bit_computing).  

![Number Types](img/num-types-solid.jpg)

Rings are [sets](https://en.wikipedia.org/wiki/Set_(mathematics)) with two operations, addition and multiplication, which allow the rationale of secret sharing and reconstruction to work. 

Plainly, secrets and secret shares **must** be integers within -2,147,483,647 to +2,147,483,647

## Governance
Additive secret sharing provides shared governance. The threshold `t` to reconstruct `s` is equal to `n`, which means **no party can recover the data** alone because all the shares are required to decrypt the secret *(t = n)*. This scheme allows us to do computation on the shares while each shareholder is only aware of their **own** share.

### [Quiz] Find the secret `s`

In practice, we use a **very high prime number** Q to add a **big deal of uniform randomness** to our shares. Here we will use a very small Q, so you can try to solve the quiz without programming yet. 

Let $ s_1 = 10 \; and \; s_2 = 74 \; and \; Q = 59 $

What is the original secret `s`? Fill the ____ space below with your answer.
Try **not** to use a calculator or programming.

In [2]:
# Run this cell to import the quizzes
from quiz import q0, q1, q2

In [3]:
# run to check your answer
s = (10 + 74)%59
q0.check(s)

Success: Correct!


True

In [4]:
# Uncomment the line below to see a hint
q0.hint

Hint: Modulo (%) is the remainder of a division 

 s = ( s₁ + s₂ ) % Q


In [5]:
# Uncomment the line below to see the solution
q0.solution

Solution: (10 + 74 ) % 59 = 25


### [Quiz] Find the final share s<sub>2</sub>

Using a small `Q` to facilitate calculation (it needs to be a **very high prime number** in production), let 

$ s = 7, n = 2 $ with $ Q = 59 $ and $ s_1 = 9 $
plugged in on the secret reconstruction equation, find the final share s<sub>2</sub>.

Fill the ____ space below with your answer. Feel free to implement the equation in a new cell or use whatever tool you'd like (e.g. a calculator), it's your call.  

In [6]:
# Fill the ____ space below with your answer
final_share = (7 - 9) % 59

# run to check your answer
q1.check(final_share)

Success: Nicely done!


True

In [7]:
# Uncomment the line below to see a hint
q1.hint

Hint: Plug the variables mentioned above into s₂ = Q - (s₁ % Q)


In [8]:
# Uncomment the line below to see the solution
q1.solution

Solution: s₂ = 57


## In Practice
Just as an educational example, we can generate a list of prime numbers using [sympy](https://www.sympy.org/en/index.html)

In [12]:
# Verify we have all the tools we need to run the notebook
!pip install -r requirements.txt
!pip install sympy

In [13]:
import sympy

# An arbitrary constant, feel free to play with it
CONST = 999

BIT_DEPTH = 31

# Range start
start = 2**BIT_DEPTH-CONST

# Maximum in Z2**32 ring
end = 2**BIT_DEPTH 

prime_lst = list(sympy.primerange(start,end+1))

print("Prime numbers in range: " , prime_lst)

Prime numbers in range:  [2147482661, 2147482663, 2147482681, 2147482693, 2147482697, 2147482739, 2147482763, 2147482801, 2147482811, 2147482817, 2147482819, 2147482859, 2147482867, 2147482873, 2147482877, 2147482921, 2147482937, 2147482943, 2147482949, 2147482951, 2147483029, 2147483033, 2147483053, 2147483059, 2147483069, 2147483077, 2147483123, 2147483137, 2147483171, 2147483179, 2147483237, 2147483249, 2147483269, 2147483323, 2147483353, 2147483399, 2147483423, 2147483477, 2147483489, 2147483497, 2147483543, 2147483549, 2147483563, 2147483579, 2147483587, 2147483629, 2147483647]


And **randomly** choose one every time using [NumPy](https://numpy.org/devdocs/contents.html)'s [randint](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html)

In [14]:
from numpy.random import randint

Q = prime_lst[randint(len(prime_lst))]
Q

2147482921

As an additional note, the [Secrets module](https://docs.python.org/3/library/secrets.html), introduced in Python 3.6, provides randomness as secure as your operating system.

In [15]:
import secrets 
  
Q = secrets.choice(prime_lst)
Q

2147483249

## The Final Share and 2-party Additive Secret Sharing

Knowing that $ s_n = Q - (\;  \sum \limits _{i=1} ^{n-1} s_i \; mod \; Q \; ) + s $

How do we implement 2-party ($ n=2 $) additive secret sharing using Python? 

Keep reading and fing out!

In [16]:
def dual_share(s, r):
    '''
    s = secret
    r = randomness
    '''
    share_lst = list()
    share_lst.append(randint(0,r))
        
    final_share = r - (share_lst[0] % r) + s
    
    share_lst.append(final_share)
        
    return share_lst

In [17]:
# Let's generate a couple of shares
secret = 5 

dual_shares = dual_share(secret, Q)
dual_shares

[292650972, 1854832282]

Now go back to the previous cell and **run it again**. Notice anything?

...

...

... 

See it yet? The shares are never the same because they are **randomly generated**.

Now let's implement the reconstruction (or decryption) function.

In [18]:
def decrypt(shares, r):
    '''
    shares = iterable made of additive secret shares
    r = randomness
    '''
    return sum(shares) % r

In [19]:
# And let's decrypt our secret for the first time
decrypt(dual_shares, Q)

5

## Exercise: Implement n-party additive secret sharing 
Fill the function below with your code.

In [20]:
def n_share(s, r, n):
    '''
    s = secret
    r = randomness
    n = number of nodes, workers or participants    
    
    returns a tuple of n-shares
    '''
    # replace with your code
    share_lst = list()
    for i in range(n-1):
        share_lst.append(randint(0,r))
        
    final_share = r - (sum(share_lst) % r) + s
    
    share_lst.append(final_share)
        
    return share_lst

In [21]:
five_shares = n_share(s=686,r=Q,n=5)
five_shares

# run this cell to check your solution
q2.check(decrypt(five_shares, Q))

Success: Nicely done!


True

In [22]:
# Uncomment the line below to see a hint
q2.hint

Hint: Iterate from i to n-1 and append shares.
            
 Also, sum all the shares before the modulo operation
            
 Plus, don't forget tuples are more secure


In [23]:
# Uncomment the line below to see the solution
q2.solution

Solution: 

                # make an iterable
                share_lst = list()


                # generate shares randomly except for the final share
                for i in range(n - 1):


                    share_lst.append(randint(0,r))



                final_share = r - (sum(share_lst) % r) + s


                share_lst.append(final_share)


                # return a tuple of shares
                return tuple(share_lst)


## Addition
Given two shared values $a$ and $b$, a party $P_i$ can compute the added shares as:
$ c_i = ( a_i + b_i ) \; mod \; Q$

In Python, we can implement this type of addition like this:

In [24]:
def addition(a, b, r):
    '''
    a = iterable of the same length of b
    b = iterable of the same length of a
    r = randomness AKA randomly generated very high prime number 
    '''
    c = list()
    
    for i in range(len(a)):
        c.append((a[i] + b[i]) % r)
        
    return tuple(c)

Considering Alice and Bob are our parties, with secrets $s_a$ and $s_b$ to be shared (2-way) and wanting to compute addition.

Let $s_a = 5 $ and $s_b = 11 $

Alice's shares would be something like:

In [25]:
# Alice's secret
sa = 5 

alice_shares = dual_share(sa, Q)
alice_shares

[264044399, 1883438855]

While Bob's shares would be

In [26]:
# Bob's secret
sb = 11

bob_shares = dual_share(sb, Q)
bob_shares

[780942505, 1366540755]

In [27]:
secret_sum = addition(alice_shares, bob_shares, Q)
secret_sum

(1044986904, 1102496361)

Doesn't make a lot of sense, does it?

Secret shares must only reveal information about their secrets when they are all combined. Otherwise all data must be hidden, which defines the **privacy** property. 

These are still secret shares so there is one more step to get the sum of the original secrets. 

In [28]:
decrypt(secret_sum, Q)

16

Et Voilà!

## Public (scalar) Multiplication
Given a list of shared values $a$ and a **scalar** $b$, a party $P_i$ can compute the multiplied shares as:
$ c_i = a_i \times b \; mod \; Q$

In Python, we can implement this type of multiplication like this:

In [29]:
def public_mul(a, b, r):
    '''
    a = iterable of the same length of b
    b = scalar to multiply a by
    r = randomness AKA randomly generated very high prime number 
    '''
    c = list()
    
    for i in range(len(a)):
        c.append((a[i] * b) % r)
        
    return tuple(c)

Let's say another party wants to multiply Alice's shares by the **scalar** value of 3. 

In [30]:
alice_times3 = public_mul(alice_shares, 3, Q)

Then we can decrypt (with Alice's permission) to double check we did multiply what we intended.

In [31]:
decrypt(alice_times3,Q)

15

And this is `True` because Alice's secret $sa = 5$, remember?

In [32]:
decrypt(alice_times3,Q) == sa * 3

True

## PyTorch + PySyft implementation
Now that you know how additive secret sharing works under the hood, let's see how we can leverage PyTorch and PySyft to do it for us.

```
pip install torch==1.4.0 -f https://download.pytorch.org/whl/torch_stable.html
pip install syft==0.2.9
```

In [1]:
import torch
import syft as sy
hook = sy.TorchHook(torch) # pip install torch==1.4.0 -f https://download.pytorch.org/whl/torch_stable.html and pip install syft==0.2.9

Let's say Alice, Bob and Charlie are all enrolled on the **Foundations of Privacy** course and we, as instructors, want to know on average, how far in the course they are. We don't want to breach their privacy so each percentage of completion will be their own secret (a, b and c). 

For educational purposes, we will define our parties (nodes, workers, etc) using `VirtualWorker` PySyft objects.

In [2]:
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
charlie = sy.VirtualWorker(hook, id="charlie")

We also need a "secure worker", also known as the `Crypto Provider` to provide us with random prime numbers.

In [3]:
secure_worker = sy.VirtualWorker(hook, "secure_worker")

We define our secrets using `torch.tensor` PyTorch tensor objects and we `Additive Share` them with our fellow workers.

In [4]:
# Let a, b and c be our students' completion percentage
a = torch.tensor([35])
b = torch.tensor([77])
c = torch.tensor([10])

In [5]:
# And we additive share with our parties
a = a.share(alice, bob, charlie, crypto_provider=secure_worker)
b = b.share(alice, bob, charlie, crypto_provider=secure_worker)
c = c.share(alice, bob, charlie, crypto_provider=secure_worker)

In [6]:
# And we compute the mean of our tensor
mean = torch.mean(torch.stack(list([a,b,c])))
mean

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:86034491782 -> alice:16724097366]
	-> [PointerTensor | me:5635845094 -> bob:88353903700]
	-> [PointerTensor | me:66858620258 -> charlie:390367811]
	*crypto provider: secure_worker*

Also, see that the object type is **[AdditiveSharingTensor]**.
For this example, we can decrypt our computation result using the get() method

In [7]:
decrypted_mean = mean.get()
decrypted_mean

tensor(40)

And get the scalar using the item() method (Only works for 1-dimensional tensors).

In [8]:
scalar_mean = decrypted_mean.item()
scalar_mean

40

Now, the average completion should actually be 40 and $ \frac{1}{3} $ (or 40.6666666666... ) but this is something we will learn about in the next lessons.

Let’s now tackle private multiplication!